In [ ]:
# prompt: connect drive and save this to '/content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/snapshot as snapshot_deployment

from google.colab import drive
import os

#mount drive if not mounted
if not os.path.exists('/content/drive'):
  drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
import requests
import json
import pandas as pd
import os
import time

def generate_query(first, skip):
    return """
        query {
          ranking(first: %s, skip: %s) {
            items {
              id
              name
              twitter
              proposalsCount
              votesCount
              followersCount
              website
            }
          }
        }
    """ % (first, skip)

out_dir = 'cached_snapshot_data_2023_07_20'
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

URL = "https://hub.snapshot.org/graphql"

def get_all_deployments():
    has_next_page = True
    skip = 0
    max_skip = 100000  # Avoid infinite loop
    step = 20

    while has_next_page and skip < max_skip:
        cache_file = f'{out_dir}/snapshot_organizations_{skip}.json'
        if os.path.exists(cache_file):
            print('Skipping cached:', skip)
            skip += step
            continue

        query = generate_query(step, skip)
        try:
            response = requests.post(URL, json={'query': query})
            if response.status_code != 200:
                print(f"HTTP error {response.status_code} at skip {skip}. Retrying...")
                time.sleep(5)
                continue

            data = response.json()

            # Handle rate limit or API error
            if 'error' in data:
                print(f"Error at skip {skip}: {data['error_description']}")
                print("Backoff before retrying...")
                time.sleep(30)
                continue

            if 'errors' in data:
                print(f"GraphQL errors: {data['errors']}")
                if 'The `skip` argument must not be greater than' in data['errors'][0].get('message', ''):
                    break
                time.sleep(10)
                continue

            # Safe access to items
            organizations = data.get('data', {}).get('ranking', {}).get('items', [])
            if not organizations:
                print(f"No organizations at skip {skip}. Ending.")
                break

            with open(cache_file, 'w') as f:
                json.dump(organizations, f)

            # print(f"Saved {len(organizations)} orgs at skip {skip}")
            skip += step
            time.sleep(1)  # Pause to avoid hitting rate limits

        except Exception as e:
            print(f"Exception at skip {skip}: {e}")
            time.sleep(10)

# Run it
get_all_deployments()


Skipping cached: 0
Skipping cached: 20
Skipping cached: 40
Skipping cached: 60
Skipping cached: 80
Skipping cached: 100
Skipping cached: 120
Skipping cached: 140
Skipping cached: 160
Skipping cached: 180
Skipping cached: 200
Skipping cached: 220
Skipping cached: 240
Skipping cached: 260
Skipping cached: 280
Skipping cached: 300
Skipping cached: 320
Skipping cached: 340
Skipping cached: 360
Skipping cached: 380
Skipping cached: 400
Skipping cached: 420
Skipping cached: 440
Skipping cached: 460
Skipping cached: 480
Skipping cached: 500
Skipping cached: 520
Skipping cached: 540
Skipping cached: 560
Skipping cached: 580
Skipping cached: 600
Skipping cached: 620
Skipping cached: 640
Skipping cached: 660
Skipping cached: 680
Skipping cached: 700
Skipping cached: 720
Skipping cached: 740
Skipping cached: 760
Skipping cached: 780
Skipping cached: 800
Skipping cached: 820
Skipping cached: 840
Skipping cached: 860
Skipping cached: 880
Skipping cached: 900
Skipping cached: 920
Skipping cached: 94

In [ ]:
# load all cached data into a df
files = os.listdir(out_dir)
files = [f for f in files if 'snapshot_organizations' in f]
dfs = []
for f in files:
    with open(f'{out_dir}/{f}', 'r') as f:
        data = json.load(f)
    dfs.append(pd.DataFrame(data))
# reset index
snapshot_deployment = pd.concat(dfs)
snapshot_deployment

,id,name,twitter,proposalsCount,votesCount,followersCount,website
0,linead87.eth,test,None,1,1,1,None
1,linead89.eth,test,None,1,1,1,None
2,linead9.eth,test,None,1,1,1,None
3,linead91.eth,test,None,1,1,1,None
4,linead93.eth,test,None,1,1,1,None
...,...,...,...,...,...,...,...
15,880010.eth,34SD,None,2,2,0,None
16,8888033.eth,8xs,None,2,2,0,None
17,8888181.eth,Vole,None,2,2,0,None
18,8889988899.eth,Lala Neto,None,2,2,0,None


In [ ]:
(snapshot_deployment['proposalsCount'] < 10).value_counts()

,count
proposalsCount,
True,68128
False,3457


In [ ]:
df = snapshot_deployment

In [ ]:
df['additional'] = df[['twitter', 'website']].apply(lambda x: ', '.join(x.dropna()), axis=1)
# drop twitter and website columns
df.drop(['twitter', 'website'], axis=1, inplace=True)
df

,id,name,proposalsCount,votesCount,followersCount,additional
0,linead87.eth,test,1,1,1,
1,linead89.eth,test,1,1,1,
2,linead9.eth,test,1,1,1,
3,linead91.eth,test,1,1,1,
4,linead93.eth,test,1,1,1,
...,...,...,...,...,...,...
15,880010.eth,34SD,2,2,0,
16,8888033.eth,8xs,2,2,0,
17,8888181.eth,Vole,2,2,0,
18,8889988899.eth,Lala Neto,2,2,0,


In [ ]:
# add platform = 'snapshot'
df['platform'] = 'snapshot'
# rename id to platform_id
df.rename(columns={'id': 'platform_id'}, inplace=True)
df

,platform_id,name,proposalsCount,votesCount,followersCount,additional,platform
0,linead87.eth,test,1,1,1,,snapshot
1,linead89.eth,test,1,1,1,,snapshot
2,linead9.eth,test,1,1,1,,snapshot
3,linead91.eth,test,1,1,1,,snapshot
4,linead93.eth,test,1,1,1,,snapshot
...,...,...,...,...,...,...,...
15,880010.eth,34SD,2,2,0,,snapshot
16,8888033.eth,8xs,2,2,0,,snapshot
17,8888181.eth,Vole,2,2,0,,snapshot
18,8889988899.eth,Lala Neto,2,2,0,,snapshot


In [ ]:
# add website column https://snapshot.org/#/ + platform_id
df['website'] = 'https://snapshot.org/#/' + df['platform_id']
df

,platform_id,name,proposalsCount,votesCount,followersCount,additional,platform,website
0,linead87.eth,test,1,1,1,,snapshot,https://snapshot.org/#/linead87.eth
1,linead89.eth,test,1,1,1,,snapshot,https://snapshot.org/#/linead89.eth
2,linead9.eth,test,1,1,1,,snapshot,https://snapshot.org/#/linead9.eth
3,linead91.eth,test,1,1,1,,snapshot,https://snapshot.org/#/linead91.eth
4,linead93.eth,test,1,1,1,,snapshot,https://snapshot.org/#/linead93.eth
...,...,...,...,...,...,...,...,...
15,880010.eth,34SD,2,2,0,,snapshot,https://snapshot.org/#/880010.eth
16,8888033.eth,8xs,2,2,0,,snapshot,https://snapshot.org/#/8888033.eth
17,8888181.eth,Vole,2,2,0,,snapshot,https://snapshot.org/#/8888181.eth
18,8889988899.eth,Lala Neto,2,2,0,,snapshot,https://snapshot.org/#/8889988899.eth


In [ ]:
# rename votesCount to votes_count and proposalsCount to proposals_count
df.rename(columns={'votesCount': 'votes_count', 'proposalsCount': 'proposals_count'}, inplace=True)
# reorder columns
df = df[['platform', 'platform_id', 'name', 'additional', 'website', 'votes_count', 'proposals_count']]
df

,platform,platform_id,name,additional,website,votes_count,proposals_count
0,snapshot,linead87.eth,test,,https://snapshot.org/#/linead87.eth,1,1
1,snapshot,linead89.eth,test,,https://snapshot.org/#/linead89.eth,1,1
2,snapshot,linead9.eth,test,,https://snapshot.org/#/linead9.eth,1,1
3,snapshot,linead91.eth,test,,https://snapshot.org/#/linead91.eth,1,1
4,snapshot,linead93.eth,test,,https://snapshot.org/#/linead93.eth,1,1
...,...,...,...,...,...,...,...
15,snapshot,880010.eth,34SD,,https://snapshot.org/#/880010.eth,2,2
16,snapshot,8888033.eth,8xs,,https://snapshot.org/#/8888033.eth,2,2
17,snapshot,8888181.eth,Vole,,https://snapshot.org/#/8888181.eth,2,2
18,snapshot,8889988899.eth,Lala Neto,,https://snapshot.org/#/8889988899.eth,2,2


In [ ]:
#create path
import os
if not os.path.exists('/content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/snapshot'):
    os.makedirs('/content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/snapshot')

# Define the path to save the file, including the filename and extension
save_path = '/content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/snapshot/snapshot_deployment1.csv'

# Save the DataFrame to a CSV file in the specified Drive location
df.to_csv(save_path, index=False)

print(f"DataFrame saved to: {save_path}")

DataFrame saved to: /content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/snapshot/snapshot_deployment1.csv


## snapshot proposal df

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/snapshot/snapshot_deployment1.csv')


In [ ]:
space_ids = df.platform_id.unique()
len(space_ids)

71585

In [ ]:
df.head()

,platform,platform_id,name,additional,website,votes_count,proposals_count
0,snapshot,linead87.eth,test,NaN,https://snapshot.org/#/linead87.eth,1,1
1,snapshot,linead89.eth,test,NaN,https://snapshot.org/#/linead89.eth,1,1
2,snapshot,linead9.eth,test,NaN,https://snapshot.org/#/linead9.eth,1,1
3,snapshot,linead91.eth,test,NaN,https://snapshot.org/#/linead91.eth,1,1
4,snapshot,linead93.eth,test,NaN,https://snapshot.org/#/linead93.eth,1,1


In [ ]:
df.proposals_count.describe()

,proposals_count
count,71585.000000
mean,3.152141
std,25.431009
min,0.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,3732.000000


In [ ]:
# remove the space_ids that have no proposals
df = df[df.proposals_count > 0]
platforms_and_proposal_count = df[['platform_id', 'proposals_count']].sort_values(by='proposals_count', ascending=False)
# make these into a list of tuples
platforms_and_proposal_count = list(platforms_and_proposal_count.itertuples(index=False, name=None))
platforms_and_proposal_count[:5]

[('cakevote.eth', 3732),
 ('snapshot.dcl.eth', 2771),
 ('cvx.eth', 1695),
 ('index-coop.eth', 1444),
 ('aladdindao.eth', 1167)]

In [ ]:
len(platforms_and_proposal_count)

63903

In [ ]:
# Make a copy before mutation
platforms_and_proposal_count_backup = platforms_and_proposal_count.copy()

In [ ]:
# make groups of rows where the proposals_count in the group sums to less than 5000
groups_of_space_ids_that_sum_to_less_than_5000 = []
while len(platforms_and_proposal_count) > 0:
    group = []
    total = 0
    while total < 5000:
        if platforms_and_proposal_count[0][1] + total > 5000:
            break
        row = platforms_and_proposal_count.pop(0)
        group.append(row)
        total += row[1]
        if len(platforms_and_proposal_count) == 0:
            break
    groups_of_space_ids_that_sum_to_less_than_5000.append(group)

groups_of_space_ids_that_sum_to_less_than_5000

[[('cakevote.eth', 3732)],
 [('snapshot.dcl.eth', 2771), ('cvx.eth', 1695)],
 [('index-coop.eth', 1444),
  ('aladdindao.eth', 1167),
  ('veclev.eth', 1111),
  ('balancer.eth', 981)],
 [('sdbal.eth', 946),
  ('bentfinance.eth', 859),
  ('sharkdao.eth', 846),
  ('aavedao.eth', 844),
  ('sdcrv-gov.eth', 822)],
 [('aurafinance.eth', 820),
  ('hvax.eth', 784),
  ('leagueoflils.eth', 680),
  ('frami.eth', 657),
  ('huwacoin.eth', 635),
  ('concentratordao.eth', 605),
  ('aavegotchi.eth', 558)],
 [('bancornetwork.eth', 555),
  ('pnounsdao.eth', 555),
  ('lemu.dcl.eth', 518),
  ('frax.eth', 499),
  ('orbapp.eth', 479),
  ('sdfxs.eth', 467),
  ('fabien.eth', 452),
  ('jbdao.eth', 412),
  ('goopsnapshot.eth', 394),
  ('alluo.eth', 389)],
 [('arbitrumfoundation.eth', 370),
  ('lido-snapshot.eth', 370),
  ('goodmorningnews.eth', 352),
  ('apecoin.eth', 326),
  ('magicappstore.eth', 323),
  ('fuse.eth', 317),
  ('decentralgames.eth', 308),
  ('olympusdao.eth', 304),
  ('futera.eth', 288),
  ('bgans

In [ ]:
# make into a list of strings for the graphql query
groups_of_space_ids_that_sum_to_less_than_5000_strings = []
for group in groups_of_space_ids_that_sum_to_less_than_5000:
    group_of_space_ids = [str(x[0]) for x in group]
    query_string = ", ".join([f'"{sid}"' for sid in group_of_space_ids])
    groups_of_space_ids_that_sum_to_less_than_5000_strings.append(query_string)

groups_of_space_ids_that_sum_to_less_than_5000_strings[0]

'"cakevote.eth"'

In [ ]:
def make_query(space_ids: str, skip: int) -> str:
    first = 1000
    base = """
        query Proposals {
         proposals(where: {space_in: [%s], state: "closed"}, first: %s, skip: %s, orderDirection: desc, orderBy: "created") {
            id
            space {
              id
            }
            author
            created
            votes
          }
        }
    """
    return base % (space_ids, first, skip)

In [ ]:
# prompt: install backoff

!pip install backoff

In [ ]:
import requests
import backoff
URL = "https://hub.snapshot.org/graphql"


@backoff.on_exception(backoff.expo,
                      requests.exceptions.RequestException)
def query_snapshot(query):
    try:
        r = requests.post(URL, json={'query': query})
    except requests.exceptions.RequestException as e:
        raise SnapshotException(f"Request failed: {str(e)}")

    if r.status_code != 200:
        raise SnapshotException(f"Non-200 status code: {r.status_code} - {r.text}")

    try:
        data = r.json()
    except Exception as e:
          raise SnapshotException(f"Failed to parse JSON: {str(e)}")

    if 'errors' in data:
        raise SnapshotException(f"GraphQL errors: {data['errors']}")

    return data

In [ ]:
total_p=0

In [ ]:
import os
import json
import time

output_dir = '/content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/snapshot'

# Optional retry function
def safe_query_snapshot(query, max_retries=5, delay=5):
    for attempt in range(max_retries):
        result = query_snapshot(query)
        if 'data' in result and 'proposals' in result['data']:
            return result
        print(f"Warning: failed attempt {attempt + 1}, retrying after {delay}s...")
        time.sleep(delay)
    print("Max retries exceeded. Returning empty result.")
    print(f'Query: {query} \nResult: {result}')
    return {'data': {'proposals': []}}

for count, space_id_group in enumerate(groups_of_space_ids_that_sum_to_less_than_5000_strings):
    skip = 0
    has_more = True

    while has_more:
        filename = os.path.join(output_dir, f'proposals_{count}_{skip}.json')
        next_filename = os.path.join(output_dir, f'proposals_{count}_{skip + 1000}.json')

        # ✅ Skip if this file already exists
        if os.path.exists(filename):
            print(f"Skipping existing file: {filename}")
            skip += 1000
            continue

        query = make_query(space_id_group, skip)
        result = safe_query_snapshot(query)  # Use the retry-safe function

        proposals = result['data']['proposals']
        if len(proposals) == 0:
            print(f'Got 0 proposals for group {count}, skip {skip}')
            break

        total_p += len(proposals)
        with open(filename, 'w') as f:
            json.dump(proposals, f)

        time.sleep(1.5)  # ⏳ Add delay to avoid rate limit

        if len(proposals) == 1000:
            skip += 1000
        else:
            has_more = False


In [ ]:
print(total_p)

224774


In [ ]:
import pandas as pd
import glob
import json
import os

# Define the folder containing your JSON files
input_dir = '/content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/snapshot/'
file_names = glob.glob(os.path.join(input_dir, '*.json'))
print(f'Total JSON files found: {len(file_names)}')

# Helper to read and normalize one file
def load_and_flatten(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)  # this is a list of proposals
    df = pd.json_normalize(data)  # this will flatten nested dicts like 'space.id'
    return df

# Read and combine all files
all_dfs = [load_and_flatten(f) for f in file_names]
df = pd.concat(all_dfs, ignore_index=True)

print(f'Total proposals loaded: {df.shape[0]}')
print(f'Columns: {df.columns.tolist()}')

# Preview
df.head()


Total JSON files found: 230
Total proposals loaded: 224774
Columns: ['id', 'author', 'created', 'votes', 'space.id']


,id,author,created,votes,space.id
0,0x67cd4655fb399e100e3db73daa503065ea94b4bdbf7a...,0x724061efDFef4a421e8be05133ad24922D07b5Bf,1708446703,21,cvx.eth
1,0x3fb10001e8967d39acbefe8fb70ef8463cbd361ba84a...,0xBB7B59Afa3A0E5Be143b8fE9C641F00c1ecB9d69,1708435570,61,snapshot.dcl.eth
2,0xc7b3461aef02eebd6c7747f24c305968d1478c7b8a06...,0xBB7B59Afa3A0E5Be143b8fE9C641F00c1ecB9d69,1708431464,44,snapshot.dcl.eth
3,0xa1ae302da637ddd527e8713792ccfa9e0bf20bc5c1e7...,0x724061efDFef4a421e8be05133ad24922D07b5Bf,1708395609,20,cvx.eth
4,0x95b846cfade4688a86b20649ef0e467fdb6d6b3cad2a...,0x724061efDFef4a421e8be05133ad24922D07b5Bf,1708301343,40,cvx.eth


In [ ]:
df.iloc[40]

,40
id,0x00d3cbf81a7e3948097f831f69d3f723bbcdd48524c6...
author,0xBB7B59Afa3A0E5Be143b8fE9C641F00c1ecB9d69
created,1707591332
votes,44
space.id,snapshot.dcl.eth


In [ ]:
df.columns

Index(['id', 'author', 'created', 'votes', 'space.id'], dtype='object')

In [ ]:
# * platform
df['platform'] = 'snapshot'
# * rename space.id to platform_deployment_id
df.rename(columns={'space.id': 'platform_deployment_id'}, inplace=True)
# * rename id to proposal_id
df.rename(columns={'id': 'proposal_id'}, inplace=True)
# * author no chg
# * rename created to date
df.rename(columns={'created': 'date'}, inplace=True)
# * votes to votes_count
df.rename(columns={'votes': 'votes_count'}, inplace=True)

# keep only these cols
df = df[['platform', 'platform_deployment_id', 'proposal_id', 'author', 'date', 'votes_count']]
df.head()

,platform,platform_deployment_id,proposal_id,author,date,votes_count
0,snapshot,cvx.eth,0x67cd4655fb399e100e3db73daa503065ea94b4bdbf7a...,0x724061efDFef4a421e8be05133ad24922D07b5Bf,1708446703,21
1,snapshot,snapshot.dcl.eth,0x3fb10001e8967d39acbefe8fb70ef8463cbd361ba84a...,0xBB7B59Afa3A0E5Be143b8fE9C641F00c1ecB9d69,1708435570,61
2,snapshot,snapshot.dcl.eth,0xc7b3461aef02eebd6c7747f24c305968d1478c7b8a06...,0xBB7B59Afa3A0E5Be143b8fE9C641F00c1ecB9d69,1708431464,44
3,snapshot,cvx.eth,0xa1ae302da637ddd527e8713792ccfa9e0bf20bc5c1e7...,0x724061efDFef4a421e8be05133ad24922D07b5Bf,1708395609,20
4,snapshot,cvx.eth,0x95b846cfade4688a86b20649ef0e467fdb6d6b3cad2a...,0x724061efDFef4a421e8be05133ad24922D07b5Bf,1708301343,40


In [ ]:
df['date'] = pd.to_datetime(df['date'], unit='s')
df.head()

,platform,platform_deployment_id,proposal_id,author,date,votes_count
0,snapshot,cvx.eth,0x67cd4655fb399e100e3db73daa503065ea94b4bdbf7a...,0x724061efDFef4a421e8be05133ad24922D07b5Bf,2024-02-20 16:31:43,21
1,snapshot,snapshot.dcl.eth,0x3fb10001e8967d39acbefe8fb70ef8463cbd361ba84a...,0xBB7B59Afa3A0E5Be143b8fE9C641F00c1ecB9d69,2024-02-20 13:26:10,61
2,snapshot,snapshot.dcl.eth,0xc7b3461aef02eebd6c7747f24c305968d1478c7b8a06...,0xBB7B59Afa3A0E5Be143b8fE9C641F00c1ecB9d69,2024-02-20 12:17:44,44
3,snapshot,cvx.eth,0xa1ae302da637ddd527e8713792ccfa9e0bf20bc5c1e7...,0x724061efDFef4a421e8be05133ad24922D07b5Bf,2024-02-20 02:20:09,20
4,snapshot,cvx.eth,0x95b846cfade4688a86b20649ef0e467fdb6d6b3cad2a...,0x724061efDFef4a421e8be05133ad24922D07b5Bf,2024-02-19 00:09:03,40


In [ ]:

# Define the path to save the file, including the filename and extension
save_path = '/content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/snapshot/snapshot_proposals1.csv'

# Save the DataFrame to a CSV file in the specified Drive location
df.to_csv(save_path, index=False)

print(f"DataFrame saved to: {save_path}")

DataFrame saved to: /content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/snapshot/snapshot_proposals1.csv


## votes

In [ ]:
import os
if not os.path.exists('votes_out_2023_07_21'):
    os.makedirs('votes_out_2023_07_21')

In [ ]:
import pandas as pd
import time
df = pd.read_csv('/content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/snapshot/snapshot_proposals1.csv')
df.head()


,platform,platform_deployment_id,proposal_id,author,date,votes_count
0,snapshot,cvx.eth,0x67cd4655fb399e100e3db73daa503065ea94b4bdbf7a...,0x724061efDFef4a421e8be05133ad24922D07b5Bf,2024-02-20 16:31:43,21
1,snapshot,snapshot.dcl.eth,0x3fb10001e8967d39acbefe8fb70ef8463cbd361ba84a...,0xBB7B59Afa3A0E5Be143b8fE9C641F00c1ecB9d69,2024-02-20 13:26:10,61
2,snapshot,snapshot.dcl.eth,0xc7b3461aef02eebd6c7747f24c305968d1478c7b8a06...,0xBB7B59Afa3A0E5Be143b8fE9C641F00c1ecB9d69,2024-02-20 12:17:44,44
3,snapshot,cvx.eth,0xa1ae302da637ddd527e8713792ccfa9e0bf20bc5c1e7...,0x724061efDFef4a421e8be05133ad24922D07b5Bf,2024-02-20 02:20:09,20
4,snapshot,cvx.eth,0x95b846cfade4688a86b20649ef0e467fdb6d6b3cad2a...,0x724061efDFef4a421e8be05133ad24922D07b5Bf,2024-02-19 00:09:03,40


In [ ]:
df.votes_count.describe()

,votes_count
count,2.247740e+05
mean,2.850903e+02
std,8.849490e+03
min,0.000000e+00
25%,1.000000e+00
50%,2.000000e+00
75%,1.000000e+01
max,2.764428e+06


In [ ]:
# sum of votes_count
df.votes_count.sum() / 5000
# we'll need this number of requests to get all votes

np.float64(12816.1786)

In [ ]:
df.sort_values(by='votes_count', ascending=False).head(10)

,platform,platform_deployment_id,proposal_id,author,date,votes_count
179971,snapshot,linea-build.eth,0xda4f201a37ea08cf1892418e7b9e88f5687a68dbdc96...,0x4214D1Abb6954daB1c61918a0CB10E006FCD33e3,2023-05-23 14:19:43,2764428
38063,snapshot,stgdao.eth,0xc18c4911e19adf4bd54c6ad142bb08bc81fd44dee723...,0xCff5be1cf3C7Cfe073Ade82d1dC27be33c998D6e,2023-08-10 15:42:20,514470
38064,snapshot,stgdao.eth,0xcfc335d45421a2d3b5f64d5ee19770ab103f539cedc2...,0xCff5be1cf3C7Cfe073Ade82d1dC27be33c998D6e,2023-08-10 15:30:36,510861
38065,snapshot,stgdao.eth,0xd6bb6427e285ae4e269ebb9f4d0396808d2de356c654...,0xCff5be1cf3C7Cfe073Ade82d1dC27be33c998D6e,2023-08-10 15:26:35,510523
37968,snapshot,stgdao.eth,0x2eff53c9e9f0bf19a98345d1d32103ec0ec859d81978...,0xd54A2392c8F2AD3a4e927Cf24EC12797f2503C7b,2023-09-04 22:05:05,472911
38351,snapshot,stgdao.eth,0xed5fe9d40c9e82a53c548d4458fdca91d6d26c12cc34...,0xCff5be1cf3C7Cfe073Ade82d1dC27be33c998D6e,2023-07-05 19:30:18,470709
37969,snapshot,stgdao.eth,0x3ab405d7d396f0b3bf04b966dd12b06ad5e81d3cf583...,0xd54A2392c8F2AD3a4e927Cf24EC12797f2503C7b,2023-09-04 21:50:33,470250
37130,snapshot,stgdao.eth,0x1c55529c8df43cc97591696a135a7b82af27fb8c58a0...,0x0E457324f0c6125b20392341Cdeb7bf9bCB02322,2024-04-24 12:38:14,467111
38003,snapshot,stgdao.eth,0xe8e92474bbd4ad0aa011cf69ad1873a9e30ab9c431f7...,0xCff5be1cf3C7Cfe073Ade82d1dC27be33c998D6e,2023-08-23 18:38:45,456653
38005,snapshot,stgdao.eth,0x6078a960e2cb6b518dddc3773f7f40b30ab76bc6f2f3...,0xCff5be1cf3C7Cfe073Ade82d1dC27be33c998D6e,2023-08-23 18:27:20,456232


In [ ]:
# remove the space_ids that have no proposals
df = df[df.votes_count > 0]
proposals_and_votes_count = df[['proposal_id', 'votes_count']].sort_values(by='votes_count', ascending=False)
# make these into a list of tuples
proposals_and_votes_count = list(proposals_and_votes_count.itertuples(index=False, name=None))
proposals_and_votes_count[:5]

[('0xda4f201a37ea08cf1892418e7b9e88f5687a68dbdc96c3ab22abaa1c7244648e',
  2764428),
 ('0xc18c4911e19adf4bd54c6ad142bb08bc81fd44dee723e520c511c2a9414d5920',
  514470),
 ('0xcfc335d45421a2d3b5f64d5ee19770ab103f539cedc240ba937691093b392bc4',
  510861),
 ('0xd6bb6427e285ae4e269ebb9f4d0396808d2de356c654c19877402da4e3e8c44a',
  510523),
 ('0x2eff53c9e9f0bf19a98345d1d32103ec0ec859d81978eecacb38db16be33b94b',
  472911)]

In [ ]:
# make large_proposals_and_votes where votes_count > 5000
large_proposals_and_votes = [x for x in proposals_and_votes_count if x[1] > 5000]
# make normal_proposals_and_votes where votes_count <= 5000
normal_proposals_and_votes = [x for x in proposals_and_votes_count if x[1] <= 5000]

In [ ]:
# save the large_proposals_and_votes
import pickle
with open('large_proposals_and_votes.pkl', 'wb') as f:
    pickle.dump(large_proposals_and_votes, f)

In [ ]:
# make groups of rows where the proposals_count in the group sums to less than 5000
groups_of_proposal_ids_that_sum_to_less_than_5000 = []
while len(normal_proposals_and_votes) > 0:
    group = []
    total = 0

    while total < 5000:
        if normal_proposals_and_votes[0][1] + total > 5000:
            break
        row = normal_proposals_and_votes.pop(0)
        group.append(row)
        total += row[1]
        if len(normal_proposals_and_votes) == 0:
            break
    groups_of_proposal_ids_that_sum_to_less_than_5000.append(group)

groups_of_proposal_ids_that_sum_to_less_than_5000

[[('0x6bccf3a1f627aa4056c921f8551683c82f3a4e0d3c831840d5738c1748f78790',
   4991)],
 [('0x643a0ee899adf7d52a5e8d4fff6040eca35d81b6c1f4cbbb557d8e721637b5aa',
   4987)],
 [('0xf922eb584e616475100222fba40b0d6574ffc113bbb5b92b76e40bb2a91e45dc',
   4966)],
 [('0x6f6ad0d74853f8d37a53bbb49c9747072b5e21d885391f5d2d4c41663dca516c',
   4963)],
 [('0x549fa2186f321a4e8a07e0d5e82f85f3e2c83189cfaee678a6df144bb67d8f54',
   4952)],
 [('0xbab5cd9d9cc5123ea5da8eb45e321463f1a0fd2543dc17587db1f24099d90dfc',
   4934)],
 [('0x707643dd2c2b941c7274f26a0362922198912e8549050ae9f2d1d9d71ada04bb',
   4918)],
 [('0x9bc6dd4fc436483309d5dc39402dc6753c9e4683ffdc3b046e3c1c1c4f8a4231',
   4880)],
 [('0xe9ff93315d1ce1ee0fe746a3074b866566de096cf912d45350f35e4fe019f502',
   4880)],
 [('0x4b362044000714197ab43d5f2155977976dcc7cf1cb28d945e1168ae7ac61f90',
   4872)],
 [('0xd17931cfa20fee418c74193df795bdca9996c2b1e95ae006d68c75ee93af163f',
   4836)],
 [('0x381c007c5d0b05adc5eb7fc5daddf3164c6d24a0c51b163046f1192f7a67fced',
   

In [ ]:
sums = [sum([x[1] for x in group]) for group in groups_of_proposal_ids_that_sum_to_less_than_5000]
sums

[4991,
 4987,
 4966,
 4963,
 4952,
 4934,
 4918,
 4880,
 4880,
 4872,
 4836,
 4814,
 4791,
 4782,
 4769,
 4716,
 4708,
 4703,
 4702,
 4699,
 4689,
 4687,
 4674,
 4665,
 4660,
 4659,
 4652,
 4652,
 4635,
 4621,
 4621,
 4618,
 4607,
 4607,
 4565,
 4552,
 4542,
 4537,
 4533,
 4531,
 4523,
 4518,
 4512,
 4502,
 4500,
 4477,
 4472,
 4468,
 4467,
 4459,
 4448,
 4435,
 4419,
 4418,
 4416,
 4406,
 4402,
 4400,
 4388,
 4368,
 4354,
 4342,
 4338,
 4329,
 4324,
 4313,
 4312,
 4296,
 4272,
 4215,
 4205,
 4194,
 4168,
 4166,
 4160,
 4155,
 4153,
 4144,
 4137,
 4129,
 4127,
 4126,
 4122,
 4121,
 4120,
 4113,
 4109,
 4103,
 4099,
 4096,
 4093,
 4088,
 4072,
 4042,
 4022,
 4010,
 3995,
 3989,
 3979,
 3979,
 3976,
 3971,
 3970,
 3969,
 3958,
 3951,
 3946,
 3944,
 3931,
 3923,
 3918,
 3891,
 3886,
 3875,
 3875,
 3848,
 3830,
 3820,
 3807,
 3804,
 3783,
 3769,
 3756,
 3746,
 3746,
 3735,
 3733,
 3732,
 3727,
 3726,
 3711,
 3706,
 3706,
 3704,
 3701,
 3701,
 3699,
 3697,
 3687,
 3673,
 3671,
 3665,
 3660,

In [ ]:
len(groups_of_proposal_ids_that_sum_to_less_than_5000)

1934

In [ ]:
# make into a list of strings for the graphql query
groups_of_proposal_ids_that_sum_to_less_than_5000_strings = []
for group in groups_of_proposal_ids_that_sum_to_less_than_5000:
    group_of_space_ids = [str(x[0]) for x in group]
    query_string = ", ".join([f'"{sid}"' for sid in group_of_space_ids])
    groups_of_proposal_ids_that_sum_to_less_than_5000_strings.append(query_string)

groups_of_proposal_ids_that_sum_to_less_than_5000_strings[0]

'"0x6bccf3a1f627aa4056c921f8551683c82f3a4e0d3c831840d5738c1748f78790"'

In [ ]:
def make_query(proposal_ids: str, skip: int) -> str:
    first = 1000
    """
    * platform
    * platform_deployment_id
    * proposal_id
    * vote_id
    * voter -- the id of the person who made the vote
    * date -- this should be as a pandas date, not as an epoch
    * choice -- this should be a string as this is not always true/false
    * weight -- the vote weight
    """
    base = """
       query {
         votes(where: {proposal_in: [%s]}, first: %s, skip: %s, orderDirection: desc, orderBy: "created") {
            space {
              id
            }
            proposal {
              id
            }
            id
            voter
            created
            vp_by_strategy
            vp
          }
        }
    """
    return base % (proposal_ids, first, skip)

In [ ]:
# prompt: install backoff

!pip install backoff

In [ ]:
import requests
import backoff
import json
import time

URL = "https://hub.snapshot.org/graphql"

class SnapshotException(Exception):
    def __init__(self, message='SnapshotException'):
        self.message = message
        super().__init__(self.message)
    def __str__(self):
        return f"SnapshotException: {self.message}"

@backoff.on_exception(
    backoff.expo,
    (requests.exceptions.RequestException, SnapshotException),
    base=2,           # base delay in seconds
    factor=2,         # exponential factor (2^n * base)
    max_value=60,     # max delay per attempt (in seconds)
    jitter=None,      # optional: add randomness to delay
    max_tries=5
)

def query_snapshot(query):
    retry_524_count = 0
    retry_429_count = 0
    max_retries = 5

    while True:
        try:
            r = requests.post(URL, json={'query': query}, timeout=60)
        except requests.exceptions.RequestException as e:
            raise SnapshotException(f"Request failed: {str(e)}")

        if r.status_code == 524:
            retry_524_count += 1
            if retry_524_count > max_retries:
                raise SnapshotException("Received HTTP 524 too many times (5). Giving up.")
            time.sleep(60)
            continue

        if r.status_code == 429:
            retry_429_count += 1
            if retry_429_count > max_retries:
                raise SnapshotException("Received HTTP 429 too many times (5). Giving up.")
            time.sleep(30)
            continue

        if r.status_code != 200:
            raise SnapshotException(f"Non-200 status code: {r.status_code} - {r.text}")

        try:
            data = r.json()
        except Exception as e:
            raise SnapshotException(f"Failed to parse JSON: {str(e)}")

        if 'errors' in data:
            raise SnapshotException(f"GraphQL errors: {data['errors']}")

        if 'data' not in data or 'votes' not in data['data']:
            raise SnapshotException(f"Missing 'votes' in response: {json.dumps(data)}")

        return data['data']['votes']

In [ ]:
# make csv log.txt in the votes_out_2023_07_21 directory if it doesn't exist
if not os.path.exists('votes_out_2023_07_21/log.csv'):
    with open('votes_out_2023_07_21/log.txt', 'w') as f:
        f.write('')

In [ ]:
import json

def process_proposal_group(count, proposal_id_group):
    # read log as list of strings
    with open('votes_out_2023_07_21/log.txt', 'r') as f:
        log = f.readlines()

    # make pagination logic
    skip = 0
    has_more = True
    votes_processed = 0
    while has_more:
        filename = f'votes_out_2023_07_21/votes_{count}_{skip}.json'
        if os.path.exists(filename) or filename + '\n' in log:
            # print('skipping', filename)
            skip += 1000
            if skip >= 5000:
                has_more = False
            continue
        try:
            query = make_query(proposal_id_group, skip)
            votes = query_snapshot(query)
        except SnapshotException as e:
            if "Read timed out" in str(e):
                print(f"⛔ Timeout for group {count}, skip {skip}. Skipping to next group.")
                return votes_processed  # Exit this group
            else:
                raise
        votes_processed += len(votes)
        time.sleep(2)  # ⏳ Add delay to avoid rate limit
        # print('got', len(votes), 'votes for', count, skip)
        if len(votes) == 0:
            print(f'Got {len(votes)} votes for group {count}, skip {skip}')

        # write filename to log
        with open('votes_out_2023_07_21/log.txt', 'a') as f:
            f.write(filename + '\n')
        # save votes to file in the output dir
        with open(filename, 'w') as f:
            json.dump(votes, f)
        if len(votes) == 1000 and skip < 5000:
            skip += 1000
        else:
            has_more = False

    return votes_processed

In [ ]:
from concurrent.futures import ThreadPoolExecutor

with ThreadPoolExecutor(max_workers=4) as executor:
    futures = [executor.submit(process_proposal_group, count, proposal_id_group) for count, proposal_id_group in enumerate(groups_of_proposal_ids_that_sum_to_less_than_5000_strings)]

    # Wait for all threads to complete
    votes_processed_list = [future.result() for future in futures]

Got 0 votes for group 510, skip 4000
Got 0 votes for group 870, skip 5000
Got 0 votes for group 871, skip 5000
Got 0 votes for group 872, skip 5000
Got 0 votes for group 873, skip 5000
Got 0 votes for group 874, skip 5000
Got 0 votes for group 875, skip 5000
Got 0 votes for group 876, skip 5000
Got 0 votes for group 877, skip 5000
Got 0 votes for group 878, skip 5000
Got 0 votes for group 879, skip 5000
Got 0 votes for group 1524, skip 5000
Got 0 votes for group 1530, skip 5000
Got 0 votes for group 1582, skip 5000
Got 0 votes for group 1583, skip 5000
Got 0 votes for group 1689, skip 5000
Got 0 votes for group 1713, skip 5000
Got 0 votes for group 1714, skip 5000
Got 0 votes for group 1715, skip 5000
Got 0 votes for group 1716, skip 5000
Got 0 votes for group 1746, skip 5000
Got 0 votes for group 1747, skip 5000
Got 0 votes for group 1800, skip 5000
Got 0 votes for group 1802, skip 5000
Got 0 votes for group 1822, skip 5000
Got 0 votes for group 1823, skip 5000
Got 0 votes for group 1

INFO:backoff:Backing off query_snapshot(...) for 2.0s (SnapshotException: SnapshotException: Request failed: HTTPSConnectionPool(host='hub.snapshot.org', port=443): Read timed out. (read timeout=60))
INFO:backoff:Backing off query_snapshot(...) for 2.0s (SnapshotException: SnapshotException: Request failed: HTTPSConnectionPool(host='hub.snapshot.org', port=443): Read timed out. (read timeout=60))
INFO:backoff:Backing off query_snapshot(...) for 2.0s (SnapshotException: SnapshotException: Request failed: HTTPSConnectionPool(host='hub.snapshot.org', port=443): Read timed out. (read timeout=60))
INFO:backoff:Backing off query_snapshot(...) for 2.0s (SnapshotException: SnapshotException: Request failed: HTTPSConnectionPool(host='hub.snapshot.org', port=443): Read timed out. (read timeout=60))
INFO:backoff:Backing off query_snapshot(...) for 4.0s (SnapshotException: SnapshotException: Request failed: HTTPSConnectionPool(host='hub.snapshot.org', port=443): Read timed out. (read timeout=60))


Got 0 votes for group 1909, skip 5000
Got 0 votes for group 1910, skip 5000
Got 0 votes for group 1911, skip 5000


INFO:backoff:Backing off query_snapshot(...) for 2.0s (SnapshotException: SnapshotException: Request failed: HTTPSConnectionPool(host='hub.snapshot.org', port=443): Read timed out. (read timeout=60))
INFO:backoff:Backing off query_snapshot(...) for 2.0s (SnapshotException: SnapshotException: Request failed: HTTPSConnectionPool(host='hub.snapshot.org', port=443): Read timed out. (read timeout=60))
INFO:backoff:Backing off query_snapshot(...) for 2.0s (SnapshotException: SnapshotException: Request failed: HTTPSConnectionPool(host='hub.snapshot.org', port=443): Read timed out. (read timeout=60))
INFO:backoff:Backing off query_snapshot(...) for 2.0s (SnapshotException: SnapshotException: Request failed: HTTPSConnectionPool(host='hub.snapshot.org', port=443): Read timed out. (read timeout=60))
INFO:backoff:Backing off query_snapshot(...) for 4.0s (SnapshotException: SnapshotException: Request failed: HTTPSConnectionPool(host='hub.snapshot.org', port=443): Read timed out. (read timeout=60))


Got 0 votes for group 1912, skip 5000


...

Got 0 votes for group 1928, skip 5000


INFO:backoff:Backing off query_snapshot(...) for 2.0s (SnapshotException: SnapshotException: Request failed: HTTPSConnectionPool(host='hub.snapshot.org', port=443): Read timed out. (read timeout=60))
INFO:backoff:Backing off query_snapshot(...) for 2.0s (SnapshotException: SnapshotException: Request failed: HTTPSConnectionPool(host='hub.snapshot.org', port=443): Read timed out. (read timeout=60))
INFO:backoff:Backing off query_snapshot(...) for 2.0s (SnapshotException: SnapshotException: Request failed: HTTPSConnectionPool(host='hub.snapshot.org', port=443): Read timed out. (read timeout=60))
INFO:backoff:Backing off query_snapshot(...) for 2.0s (SnapshotException: SnapshotException: Request failed: HTTPSConnectionPool(host='hub.snapshot.org', port=443): Read timed out. (read timeout=60))
INFO:backoff:Backing off query_snapshot(...) for 4.0s (SnapshotException: SnapshotException: Request failed: HTTPSConnectionPool(host='hub.snapshot.org', port=443): Read timed out. (read timeout=60))


Got 0 votes for group 1930, skip 5000
Got 0 votes for group 1929, skip 5000
Got 0 votes for group 1931, skip 5000


INFO:backoff:Backing off query_snapshot(...) for 2.0s (SnapshotException: SnapshotException: Request failed: HTTPSConnectionPool(host='hub.snapshot.org', port=443): Read timed out. (read timeout=60))
INFO:backoff:Backing off query_snapshot(...) for 4.0s (SnapshotException: SnapshotException: Request failed: HTTPSConnectionPool(host='hub.snapshot.org', port=443): Read timed out. (read timeout=60))
INFO:backoff:Backing off query_snapshot(...) for 2.0s (SnapshotException: SnapshotException: Request failed: HTTPSConnectionPool(host='hub.snapshot.org', port=443): Read timed out. (read timeout=60))
INFO:backoff:Backing off query_snapshot(...) for 4.0s (SnapshotException: SnapshotException: Request failed: HTTPSConnectionPool(host='hub.snapshot.org', port=443): Read timed out. (read timeout=60))
INFO:backoff:Backing off query_snapshot(...) for 2.0s (SnapshotException: SnapshotException: Request failed: HTTPSConnectionPool(host='hub.snapshot.org', port=443): Read timed out. (read timeout=60))


Got 0 votes for group 1932, skip 5000


In [ ]:
# get all the json files in votes_out_2023_07_21
import glob
file_names = glob.glob('votes_out_2023_07_21/*.json')
len(file_names)

9129

In [ ]:
# get last 32 the json files in votes_out_2023_07_21
import glob
file_names = glob.glob('votes_out_2023_07_21/*.json')
len(file_names)

In [ ]:
import json
import pandas as pd
import simplejson as json

pd.io.json._json.loads = lambda s, *a, **kw: json.loads(s)

# monkeypatch using faster simplejson module
import simplejson
pd.io.json._json.loads = lambda s, *a, **kw: simplejson.loads(s)

# normalising (unnesting) at the same time (for nested jsons)
pd.io.json._json.loads = lambda s, *a, **kw: pd.json_normalize(simplejson.loads(s))

In [ ]:
import pandas as pd
import simplejson as json

dfs = []
for f in file_names:
    with open(f) as fp:
        try:
            data = json.load(fp)
            df_part = pd.json_normalize(data)
            dfs.append(df_part)
        except Exception as e:
            print(f"Error reading {f}: {e}")

df = pd.concat(dfs, ignore_index=True)


In [ ]:
df

,id,voter,created,vp_by_strategy,vp,space.id,proposal.id
0,0x03177a05545e235947454e328fc8afd5410f1f0d15aa...,0xCfB13A6e985A52E227fc3eCc854Dd83d83Ea8CBA,1740301884,[56.83498092789634],56.834981,arbitrumfoundation.eth,0xebbcad364ae7f02797b9cd7f30c668907d479d4bb5ba...
1,0x712f77bbea15e2ef4b5c410a4978a4d890952388c0e0...,0x36af23887d371589a7C8470e098A39E6311A292f,1740301772,[4.477604993844906],4.477605,arbitrumfoundation.eth,0xebbcad364ae7f02797b9cd7f30c668907d479d4bb5ba...
2,0xc180e2e883eb03555aae434d048b43ae4ca577872dd3...,0x22a57A3f99C61B7D744f26057f5dDf2E345a6f5c,1740301623,[8.328430575448769],8.328431,arbitrumfoundation.eth,0xebbcad364ae7f02797b9cd7f30c668907d479d4bb5ba...
3,0x4bfcafc141db0f22e95aa36eb32b7d1f495d31792157...,0xD522562eC491F90cfA772185f75314901DbD591a,1740301603,[5.168611375542766],5.168611,arbitrumfoundation.eth,0xebbcad364ae7f02797b9cd7f30c668907d479d4bb5ba...
4,0x122e03cedac28eb1d80c570df520cd85285adae0e688...,0x627392B6F91D7F42A723d32059c6187cAb9f623C,1740301294,[3.175205892914577],3.175206,arbitrumfoundation.eth,0xebbcad364ae7f02797b9cd7f30c668907d479d4bb5ba...
...,...,...,...,...,...,...,...
8495502,0x1874bb84fe7eced31dcc132627c82ab8dd173580bf3d...,0x02fA80982A53cc9435691993AD9C3c41a0eE7f1A,1695666666,"[3.3054611871349104, 0]",3.305461,speraxdao.eth,0xf565e4ff580e7fee83695fbe595560a586226eeef532...
8495503,0xefeab7dc776b647a79ce0c260791431ebbc41b701c30...,0xf64E27dBA2D1686BEa22406FE5D0AC9f368De5EC,1695666666,"[9.20197146677879, 0]",9.201971,speraxdao.eth,0xf565e4ff580e7fee83695fbe595560a586226eeef532...
8495504,0xa689fc775bd129bffb3bcb143df35e759bdbaadca351...,0xb50fAB23cf6FAF33f273759f654268cEB580d2c7,1695666655,"[25.498957176469855, 0]",25.498957,speraxdao.eth,0xf565e4ff580e7fee83695fbe595560a586226eeef532...
8495505,0x5b4016516e429829949cb715472fa18045646c647ef4...,0x10EaadC21A7ff7D7aEFc7B1Fa55671705975032E,1695666640,"[25.166371545331025, 0]",25.166372,speraxdao.eth,0xf565e4ff580e7fee83695fbe595560a586226eeef532...


In [ ]:
df.head()

,id,voter,created,vp_by_strategy,vp,space.id,proposal.id
0,0x03177a05545e235947454e328fc8afd5410f1f0d15aa...,0xCfB13A6e985A52E227fc3eCc854Dd83d83Ea8CBA,1740301884,[56.83498092789634],56.834981,arbitrumfoundation.eth,0xebbcad364ae7f02797b9cd7f30c668907d479d4bb5ba...
1,0x712f77bbea15e2ef4b5c410a4978a4d890952388c0e0...,0x36af23887d371589a7C8470e098A39E6311A292f,1740301772,[4.477604993844906],4.477605,arbitrumfoundation.eth,0xebbcad364ae7f02797b9cd7f30c668907d479d4bb5ba...
2,0xc180e2e883eb03555aae434d048b43ae4ca577872dd3...,0x22a57A3f99C61B7D744f26057f5dDf2E345a6f5c,1740301623,[8.328430575448769],8.328431,arbitrumfoundation.eth,0xebbcad364ae7f02797b9cd7f30c668907d479d4bb5ba...
3,0x4bfcafc141db0f22e95aa36eb32b7d1f495d31792157...,0xD522562eC491F90cfA772185f75314901DbD591a,1740301603,[5.168611375542766],5.168611,arbitrumfoundation.eth,0xebbcad364ae7f02797b9cd7f30c668907d479d4bb5ba...
4,0x122e03cedac28eb1d80c570df520cd85285adae0e688...,0x627392B6F91D7F42A723d32059c6187cAb9f623C,1740301294,[3.175205892914577],3.175206,arbitrumfoundation.eth,0xebbcad364ae7f02797b9cd7f30c668907d479d4bb5ba...


In [ ]:
# * platform
df['platform'] = 'snapshot'
# * rename space.id to platform_deployment_id
df.rename(columns={'space.id': 'platform_deployment_id'}, inplace=True)
# * rename proposal.id to proposal_id
df.rename(columns={'proposal.id': 'proposal_id'}, inplace=True)
# * rename id to vote_id
df.rename(columns={'id': 'vote_id'}, inplace=True)
# * voter -- the id of the person who made the vote, no chg
# * rename created to date -- this should be as a pandas date, not as an epoch
df.rename(columns={'created': 'date'}, inplace=True)
# * rename vp_by_strategy to choice -- this should be a string as this is not always true/false
df.rename(columns={'vp_by_strategy': 'choice'}, inplace=True)
# make string
df['choice'] = df['choice'].astype(str)
# * rename vp to weight -- the vote weight
df.rename(columns={'vp': 'weight'}, inplace=True)
# make int
df['weight'] = df['weight'].apply(int)

df.head()

,vote_id,voter,date,choice,weight,platform_deployment_id,proposal_id,platform
0,0x03177a05545e235947454e328fc8afd5410f1f0d15aa...,0xCfB13A6e985A52E227fc3eCc854Dd83d83Ea8CBA,1740301884,[56.83498092789634],56,arbitrumfoundation.eth,0xebbcad364ae7f02797b9cd7f30c668907d479d4bb5ba...,snapshot
1,0x712f77bbea15e2ef4b5c410a4978a4d890952388c0e0...,0x36af23887d371589a7C8470e098A39E6311A292f,1740301772,[4.477604993844906],4,arbitrumfoundation.eth,0xebbcad364ae7f02797b9cd7f30c668907d479d4bb5ba...,snapshot
2,0xc180e2e883eb03555aae434d048b43ae4ca577872dd3...,0x22a57A3f99C61B7D744f26057f5dDf2E345a6f5c,1740301623,[8.328430575448769],8,arbitrumfoundation.eth,0xebbcad364ae7f02797b9cd7f30c668907d479d4bb5ba...,snapshot
3,0x4bfcafc141db0f22e95aa36eb32b7d1f495d31792157...,0xD522562eC491F90cfA772185f75314901DbD591a,1740301603,[5.168611375542766],5,arbitrumfoundation.eth,0xebbcad364ae7f02797b9cd7f30c668907d479d4bb5ba...,snapshot
4,0x122e03cedac28eb1d80c570df520cd85285adae0e688...,0x627392B6F91D7F42A723d32059c6187cAb9f623C,1740301294,[3.175205892914577],3,arbitrumfoundation.eth,0xebbcad364ae7f02797b9cd7f30c668907d479d4bb5ba...,snapshot


In [ ]:
# make date a pandas date, parse epoch timestamp type 's'
df['date'] = pd.to_datetime(df['date'], unit='s')
df.head()

,vote_id,voter,date,choice,weight,platform_deployment_id,proposal_id,platform
0,0x03177a05545e235947454e328fc8afd5410f1f0d15aa...,0xCfB13A6e985A52E227fc3eCc854Dd83d83Ea8CBA,2025-02-23 09:11:24,[56.83498092789634],56,arbitrumfoundation.eth,0xebbcad364ae7f02797b9cd7f30c668907d479d4bb5ba...,snapshot
1,0x712f77bbea15e2ef4b5c410a4978a4d890952388c0e0...,0x36af23887d371589a7C8470e098A39E6311A292f,2025-02-23 09:09:32,[4.477604993844906],4,arbitrumfoundation.eth,0xebbcad364ae7f02797b9cd7f30c668907d479d4bb5ba...,snapshot
2,0xc180e2e883eb03555aae434d048b43ae4ca577872dd3...,0x22a57A3f99C61B7D744f26057f5dDf2E345a6f5c,2025-02-23 09:07:03,[8.328430575448769],8,arbitrumfoundation.eth,0xebbcad364ae7f02797b9cd7f30c668907d479d4bb5ba...,snapshot
3,0x4bfcafc141db0f22e95aa36eb32b7d1f495d31792157...,0xD522562eC491F90cfA772185f75314901DbD591a,2025-02-23 09:06:43,[5.168611375542766],5,arbitrumfoundation.eth,0xebbcad364ae7f02797b9cd7f30c668907d479d4bb5ba...,snapshot
4,0x122e03cedac28eb1d80c570df520cd85285adae0e688...,0x627392B6F91D7F42A723d32059c6187cAb9f623C,2025-02-23 09:01:34,[3.175205892914577],3,arbitrumfoundation.eth,0xebbcad364ae7f02797b9cd7f30c668907d479d4bb5ba...,snapshot


In [ ]:
df = df[['platform', 'platform_deployment_id', 'proposal_id', 'vote_id', 'voter', 'date', 'choice', 'weight']]
df.head()

,platform,platform_deployment_id,proposal_id,vote_id,voter,date,choice,weight
0,snapshot,arbitrumfoundation.eth,0xebbcad364ae7f02797b9cd7f30c668907d479d4bb5ba...,0x03177a05545e235947454e328fc8afd5410f1f0d15aa...,0xCfB13A6e985A52E227fc3eCc854Dd83d83Ea8CBA,2025-02-23 09:11:24,[56.83498092789634],56
1,snapshot,arbitrumfoundation.eth,0xebbcad364ae7f02797b9cd7f30c668907d479d4bb5ba...,0x712f77bbea15e2ef4b5c410a4978a4d890952388c0e0...,0x36af23887d371589a7C8470e098A39E6311A292f,2025-02-23 09:09:32,[4.477604993844906],4
2,snapshot,arbitrumfoundation.eth,0xebbcad364ae7f02797b9cd7f30c668907d479d4bb5ba...,0xc180e2e883eb03555aae434d048b43ae4ca577872dd3...,0x22a57A3f99C61B7D744f26057f5dDf2E345a6f5c,2025-02-23 09:07:03,[8.328430575448769],8
3,snapshot,arbitrumfoundation.eth,0xebbcad364ae7f02797b9cd7f30c668907d479d4bb5ba...,0x4bfcafc141db0f22e95aa36eb32b7d1f495d31792157...,0xD522562eC491F90cfA772185f75314901DbD591a,2025-02-23 09:06:43,[5.168611375542766],5
4,snapshot,arbitrumfoundation.eth,0xebbcad364ae7f02797b9cd7f30c668907d479d4bb5ba...,0x122e03cedac28eb1d80c570df520cd85285adae0e688...,0x627392B6F91D7F42A723d32059c6187cAb9f623C,2025-02-23 09:01:34,[3.175205892914577],3


In [ ]:
# prompt: save this to csv

# Define the path to save the file, including the filename and extension
save_path = '/content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/snapshot/snapshot_votes2.csv'

# Save the DataFrame to a CSV file in the specified Drive location
df.to_csv(save_path, index=False)

print(f"DataFrame saved to: {save_path}")

DataFrame saved to: /content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/snapshot/snapshot_votes2.csv


In [ ]:
import pandas as pd

snapshot_votes = pd.read_csv('/content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/snapshot/snapshot_votes2.csv')
snapshot_votes.head()

<ipython-input-1-6a9430c6f813>:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  snapshot_votes = pd.read_csv('/content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/snapshot/snapshot_votes2.csv')


,platform,platform_deployment_id,proposal_id,vote_id,voter,date,choice,weight
0,snapshot,arbitrumfoundation.eth,0xebbcad364ae7f02797b9cd7f30c668907d479d4bb5ba...,0x03177a05545e235947454e328fc8afd5410f1f0d15aa...,0xCfB13A6e985A52E227fc3eCc854Dd83d83Ea8CBA,2025-02-23 09:11:24,[56.83498092789634],56
1,snapshot,arbitrumfoundation.eth,0xebbcad364ae7f02797b9cd7f30c668907d479d4bb5ba...,0x712f77bbea15e2ef4b5c410a4978a4d890952388c0e0...,0x36af23887d371589a7C8470e098A39E6311A292f,2025-02-23 09:09:32,[4.477604993844906],4
2,snapshot,arbitrumfoundation.eth,0xebbcad364ae7f02797b9cd7f30c668907d479d4bb5ba...,0xc180e2e883eb03555aae434d048b43ae4ca577872dd3...,0x22a57A3f99C61B7D744f26057f5dDf2E345a6f5c,2025-02-23 09:07:03,[8.328430575448769],8
3,snapshot,arbitrumfoundation.eth,0xebbcad364ae7f02797b9cd7f30c668907d479d4bb5ba...,0x4bfcafc141db0f22e95aa36eb32b7d1f495d31792157...,0xD522562eC491F90cfA772185f75314901DbD591a,2025-02-23 09:06:43,[5.168611375542766],5
4,snapshot,arbitrumfoundation.eth,0xebbcad364ae7f02797b9cd7f30c668907d479d4bb5ba...,0x122e03cedac28eb1d80c570df520cd85285adae0e688...,0x627392B6F91D7F42A723d32059c6187cAb9f623C,2025-02-23 09:01:34,[3.175205892914577],3


In [ ]:
snapshot_deployment = pd.read_csv('/content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/snapshot/snapshot_deployment1.csv')
snapshot_deployment.head()

,platform,platform_id,name,additional,website,votes_count,proposals_count
0,snapshot,linead87.eth,test,NaN,https://snapshot.org/#/linead87.eth,1,1
1,snapshot,linead89.eth,test,NaN,https://snapshot.org/#/linead89.eth,1,1
2,snapshot,linead9.eth,test,NaN,https://snapshot.org/#/linead9.eth,1,1
3,snapshot,linead91.eth,test,NaN,https://snapshot.org/#/linead91.eth,1,1
4,snapshot,linead93.eth,test,NaN,https://snapshot.org/#/linead93.eth,1,1


In [ ]:
# Step 1: Count unique voters per platform_deployment_id
unique_voter_counts = snapshot_votes.groupby('platform_deployment_id')['voter'].nunique()

# Step 2: Merge this with snapshot_deployment
snapshot_deployment = snapshot_deployment.merge(
    unique_voter_counts.rename('unique_voters'),
    how='left',
    left_on='platform_id',
    right_index=True
)

# Step 3: Fill NaNs (if any deployments had no votes)
snapshot_deployment['unique_voters'] = snapshot_deployment['unique_voters'].fillna(0).astype(int)


In [ ]:
snapshot_deployment

,platform,platform_id,name,additional,website,votes_count,proposals_count,unique_voters
0,snapshot,linead87.eth,test,NaN,https://snapshot.org/#/linead87.eth,1,1,1
1,snapshot,linead89.eth,test,NaN,https://snapshot.org/#/linead89.eth,1,1,1
2,snapshot,linead9.eth,test,NaN,https://snapshot.org/#/linead9.eth,1,1,1
3,snapshot,linead91.eth,test,NaN,https://snapshot.org/#/linead91.eth,1,1,1
4,snapshot,linead93.eth,test,NaN,https://snapshot.org/#/linead93.eth,1,1,1
...,...,...,...,...,...,...,...,...
71580,snapshot,880010.eth,34SD,NaN,https://snapshot.org/#/880010.eth,2,2,1
71581,snapshot,8888033.eth,8xs,NaN,https://snapshot.org/#/8888033.eth,2,2,1
71582,snapshot,8888181.eth,Vole,NaN,https://snapshot.org/#/8888181.eth,2,2,1
71583,snapshot,8889988899.eth,Lala Neto,NaN,https://snapshot.org/#/8889988899.eth,2,2,1


In [ ]:
# prompt: set platform_id as deployment_id

snapshot_deployment.rename(columns={'platform_id': 'deployment_id'}, inplace=True)


In [ ]:
# prompt: save it

# Define the path to save the file, including the filename and extension
save_path = '/content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/snapshot/snapshot_deployment.csv'

# Save the DataFrame to a CSV file in the specified Drive location
snapshot_deployment.to_csv(save_path, index=False)

print(f"DataFrame saved to: {save_path}")

DataFrame saved to: /content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/snapshot/snapshot_deployment.csv


In [ ]:
# prompt: can you find the platform_id that has a unique_voter number of greater than 1? print the number of voter that platform_id has

# Step 4: Filter for platform_id with unique_voter > 1
platforms_with_more_than_one_unique_voter = snapshot_deployment[snapshot_deployment['unique_voters'] > 1]

# Step 5: Print the platform_id and the number of unique voters
print("Platform IDs with more than 1 unique voter and their counts:")
for index, row in platforms_with_more_than_one_unique_voter.iterrows():
    print(f"Platform ID: {row['deployment_id']}, Unique Voters: {row['unique_voters']}")

Streaming output truncated to the last 5000 lines.
Platform ID: kittystage1.eth, Unique Voters: 11
Platform ID: klimen.eth, Unique Voters: 4
Platform ID: kogemaru.eth, Unique Voters: 4
Platform ID: kryptans.eth, Unique Voters: 3
Platform ID: lamborghiniegoista.eth, Unique Voters: 3
Platform ID: ledgerweb3.eth, Unique Voters: 5
Platform ID: matasaburou.eth, Unique Voters: 4
Platform ID: metamask-web3.eth, Unique Voters: 4
Platform ID: mygoldmine.eth, Unique Voters: 6
Platform ID: opiod.eth, Unique Voters: 4
Platform ID: 4a40711.eth, Unique Voters: 2
Platform ID: aironas.eth, Unique Voters: 2
Platform ID: airrrrd4.eth, Unique Voters: 2
Platform ID: aishani.eth, Unique Voters: 2
Platform ID: aitkens.eth, Unique Voters: 2
Platform ID: aiur.eth, Unique Voters: 2
Platform ID: ajanaku.eth, Unique Voters: 2
Platform ID: akenji.eth, Unique Voters: 2
Platform ID: aki18.eth, Unique Voters: 2
Platform ID: akinseker.eth, Unique Voters: 2
Platform ID: akisalehi.eth, Unique Voters: 2
Platform ID: ako

In [ ]:
import pandas as pd
import numpy as np

# Define size bins and labels
bins = [0,2, 10, 100, 1_000, 10_000, 100_000, 1_000_000, 10_000_000]
labels = ['[0,2]','[2,10]', '(10,100]', '(100,1K]', '(1K,10K]', '(10K,100K]', '(100K,1M]', '(1M,10M]']

# Filter DAOs with at least 2 unique voters
filtered_df = snapshot_deployment[snapshot_deployment["unique_voters"] >= 0].copy()

# Assign DAO size category
filtered_df["size_category"] = pd.cut(filtered_df["unique_voters"], bins=bins, labels=labels, right=True, include_lowest=True)

# Group by platform and size category
table = filtered_df.groupby(["size_category", "platform"]).size().unstack(fill_value=0)

# Add total per row (size category) and column (platform)
table["Total"] = table.sum(axis=1)
table.loc["Total"] = table.sum(axis=0)

# Reset index for display (optional)
table = table.reset_index()

# Expand display width to avoid line breaking
pd.set_option('display.width', 200)
pd.set_option('display.max_columns', None)

# Save or display
print(table)


platform size_category  snapshot  Total
0                [0,2]     58078  58078
1               [2,10]      6712   6712
2             (10,100]      4713   4713
3             (100,1K]      1780   1780
4             (1K,10K]       281    281
5           (10K,100K]        20     20
6            (100K,1M]         1      1
7             (1M,10M]         0      0
8                Total     71585  71585


<ipython-input-8-81ea96dce405>:15: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  table = filtered_df.groupby(["size_category", "platform"]).size().unstack(fill_value=0)


In [ ]:
# prompt: get snapshot_proposals data

import pandas as pd
# load the snapshot_proposals data
snapshot_proposals = pd.read_csv('/content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/snapshot/snapshot_proposals1.csv')
snapshot_proposals.head()

,platform,platform_deployment_id,proposal_id,author,date,votes_count
0,snapshot,cvx.eth,0x67cd4655fb399e100e3db73daa503065ea94b4bdbf7a...,0x724061efDFef4a421e8be05133ad24922D07b5Bf,2024-02-20 16:31:43,21
1,snapshot,snapshot.dcl.eth,0x3fb10001e8967d39acbefe8fb70ef8463cbd361ba84a...,0xBB7B59Afa3A0E5Be143b8fE9C641F00c1ecB9d69,2024-02-20 13:26:10,61
2,snapshot,snapshot.dcl.eth,0xc7b3461aef02eebd6c7747f24c305968d1478c7b8a06...,0xBB7B59Afa3A0E5Be143b8fE9C641F00c1ecB9d69,2024-02-20 12:17:44,44
3,snapshot,cvx.eth,0xa1ae302da637ddd527e8713792ccfa9e0bf20bc5c1e7...,0x724061efDFef4a421e8be05133ad24922D07b5Bf,2024-02-20 02:20:09,20
4,snapshot,cvx.eth,0x95b846cfade4688a86b20649ef0e467fdb6d6b3cad2a...,0x724061efDFef4a421e8be05133ad24922D07b5Bf,2024-02-19 00:09:03,40


In [ ]:
# prompt: rename platform_deployment_id as deployment_id and save it

snapshot_proposals.rename(columns={'platform_deployment_id': 'deployment_id'}, inplace=True)

# Define the path to save the file, including the filename and extension
save_path = '/content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/snapshot/snapshot_proposals.csv'

# Save the DataFrame to a CSV file in the specified Drive location
snapshot_proposals.to_csv(save_path, index=False)

print(f"DataFrame saved to: {save_path}")

DataFrame saved to: /content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/snapshot/snapshot_proposals.csv


In [ ]:
import pandas as pd
import numpy as np

# Define size bins and labels
bins = [2, 10, 100, 1_000, 10_000, 100_000, 1_000_000, 10_000_000]
labels = ['[2,10]', '(10,100]', '(100,1K]', '(1K,10K]', '(10K,100K]', '(100K,1M]', '(1M,10M]']

# Filter DAOs with at least 2 unique voters
filtered_df = snapshot_deployment[snapshot_deployment["unique_voters"] >= 2].copy()

# Assign DAO size category
filtered_df["size_category"] = pd.cut(filtered_df["unique_voters"], bins=bins, labels=labels, right=True, include_lowest=True)

# Group by platform and size category
table = filtered_df.groupby(["size_category", "platform"]).size().unstack(fill_value=0)

# Add total per row (size category) and column (platform)
table["Total"] = table.sum(axis=1)
table.loc["Total"] = table.sum(axis=0)

# Reset index for display (optional)
table = table.reset_index()

# Expand display width to avoid line breaking
pd.set_option('display.width', 200)
pd.set_option('display.max_columns', None)

# Save or display
print(table)

platform size_category  snapshot  Total
0               [2,10]     14575  14575
1             (10,100]      4713   4713
2             (100,1K]      1780   1780
3             (1K,10K]       281    281
4           (10K,100K]        20     20
5            (100K,1M]         1      1
6             (1M,10M]         0      0
7                Total     21370  21370


<ipython-input-15-1019e6a97918>:15: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  table = filtered_df.groupby(["size_category", "platform"]).size().unstack(fill_value=0)
